In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import lightgbm as lgb

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import json

ROOT = Path('.')
TRAIN_CSV = ROOT / 'train_air_respiratory.csv'
TEST_CSV = ROOT / 'test_air_respiratory.csv'
COUGH_DIR = ROOT / 'dataclean_cough_mfcc'
VOWEL_DIR = ROOT / 'dataclean_vowel_mfcc'

# Helper to mean-pool MFCC from JSON (handles dict structure with 'features'->'mfcc')
def load_mean_embedding_json(folder: Path):
    mfcc_path = folder / 'mfcc_features.json'
    if not mfcc_path.exists():
        return None
    try:
        with open(mfcc_path, 'r') as f:
            data = json.load(f)
        if isinstance(data, dict) and 'features' in data and 'mfcc' in data['features']:
            arr = np.asarray(data['features']['mfcc'], dtype=np.float32)
        elif isinstance(data, dict) and 'mfcc' in data:
            arr = np.asarray(data['mfcc'], dtype=np.float32)
        elif isinstance(data, list):
            arr = np.asarray(data, dtype=np.float32)
        else:
            return None
        if arr.ndim == 2:
            emb = arr.mean(axis=0)
        elif arr.ndim == 1:
            emb = arr
        else:
            return None
        print(f"Loaded {mfcc_path} embedding dim {emb.shape[0]}")
        return emb
    except Exception as e:
        print(f"Error reading {mfcc_path}: {e}")
        return None

train_df = pd.read_csv(TRAIN_CSV).drop_duplicates(subset=['candidateID'])
label_map = dict(zip(train_df['candidateID'], train_df['disease']))

cough_X, cough_y, vowel_X, vowel_y = [], [], [], []
missing_cough, missing_vowel = 0, 0

for cid, label in label_map.items():
    cough_folder = COUGH_DIR / cid
    cough_emb = load_mean_embedding_json(cough_folder)
    if cough_emb is not None:
        cough_X.append(cough_emb)
        cough_y.append(label)
    else:
        missing_cough += 1

    vowel_folder = VOWEL_DIR / cid
    vowel_emb = load_mean_embedding_json(vowel_folder)
    if vowel_emb is not None:
        vowel_X.append(vowel_emb)
        vowel_y.append(label)
    else:
        missing_vowel += 1

if cough_X:
    cough_X = np.vstack(cough_X)
else:
    cough_X = np.empty((0, 0))
if vowel_X:
    vowel_X = np.vstack(vowel_X)
else:
    vowel_X = np.empty((0, 0))
cough_y = np.array(cough_y)
vowel_y = np.array(vowel_y)

print(f"Cough specialist samples: {len(cough_y)} (missing/skipped: {missing_cough})")
print(f"Vowel specialist samples: {len(vowel_y)} (missing/skipped: {missing_vowel})")

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

base_params = dict(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=400,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multiclass',    
    metric='multi_logloss',    
    num_class=3,
    n_jobs=4,
    random_state=42,
    verbose=-1      )

def train_with_eval(X, y, name: str):
    if len(y) == 0 or X.shape[0] == 0 or X.shape[1] == 0:
        print(f"{name}: no data, training skipped")
        return None
    unique_classes = np.unique(y)
    can_eval = len(y) >= 10 and len(unique_classes) >= 2
    if can_eval:
        stratify = y if len(unique_classes) > 1 else None
        X_tr, X_val, y_tr, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=stratify
        )
        model = lgb.LGBMClassifier(**base_params)
        model.fit(X_tr, y_tr)
        val_pred = model.predict(X_val)
        metrics = {
            'f1_macro': f1_score(y_val, val_pred, average='macro'),
            'accuracy': accuracy_score(y_val, val_pred),
            'precision_macro': precision_score(y_val, val_pred, average='macro', zero_division=0),
            'recall_macro': recall_score(y_val, val_pred, average='macro'),
        }
        print(f"\n{name} validation metrics (holdout 20%):")
        for k, v in metrics.items():
            print(f"  {k}: {v:.4f}")
        print(classification_report(y_val, val_pred, target_names=['Healthy', 'Asthma', 'COPD']))
    else:
        print(f"{name}: skipped holdout metrics (samples={len(y)}, classes={len(unique_classes)})")
    final_model = lgb.LGBMClassifier(**base_params)
    final_model.fit(X, y)
    return final_model

cough_model = train_with_eval(cough_X, cough_y, "Cough specialist")
vowel_model = train_with_eval(vowel_X, vowel_y, "Vowel specialist")

test_df = pd.read_csv(TEST_CSV)
preds, sources = [], []

for cid in test_df['candidateID']:
    cough_folder = COUGH_DIR / cid
    vowel_folder = VOWEL_DIR / cid
    options = []

    cough_emb = load_mean_embedding_json(cough_folder)
    if cough_emb is not None and cough_model is not None:
        prob = cough_model.predict_proba(cough_emb.reshape(1, -1))[0]
        options.append((prob, 'cough'))

    vowel_emb = load_mean_embedding_json(vowel_folder)
    if vowel_emb is not None and vowel_model is not None:
        prob = vowel_model.predict_proba(vowel_emb.reshape(1, -1))[0]
        options.append((prob, 'vowel'))

    if options:
        best_prob, best_src = max(options, key=lambda item: item[0].max())
        pred_class = int(best_prob.argmax())
    else:
        best_src = 'fallback'
        pred_class = 2  # fallback to COPD

    preds.append(pred_class)
    sources.append(best_src)

submission = pd.DataFrame({'candidateID': test_df['candidateID'], 'disease': preds})
submission_path = ROOT / 'submission_lightgbmjere_dual_audio.csv'
submission.to_csv(submission_path, index=False)

print(f"Saved submission to {submission_path} with {len(submission)} rows")
print("Prediction source counts:\n", pd.Series(sources).value_counts())


Loaded dataclean_cough_mfcc\b87ea0dd760fa\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\b87ea0dd760fa\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\b734834dec6d1\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\b734834dec6d1\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\e000a41725f53\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\e000a41725f53\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\253ba780a0398\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\253ba780a0398\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\6663cd023ff3d\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\6663cd023ff3d\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\5d5010f2cc0fd\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\58ee9523cbcbc\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\58ee9523cbcbc\mfcc_features.json emb

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Cough specialist validation metrics (holdout 20%):
  f1_macro: 0.2809
  accuracy: 0.3426
  precision_macro: 0.2743
  recall_macro: 0.2986
              precision    recall  f1-score   support

     Healthy       0.28      0.25      0.26        28
      Asthma       0.44      0.58      0.50        48
        COPD       0.11      0.06      0.08        32

    accuracy                           0.34       108
   macro avg       0.27      0.30      0.28       108
weighted avg       0.30      0.34      0.31       108



c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Vowel specialist validation metrics (holdout 20%):
  f1_macro: 0.3407
  accuracy: 0.3645
  precision_macro: 0.3422
  recall_macro: 0.3399
              precision    recall  f1-score   support

     Healthy       0.28      0.26      0.27        27
      Asthma       0.49      0.48      0.48        48
        COPD       0.26      0.28      0.27        32

    accuracy                           0.36       107
   macro avg       0.34      0.34      0.34       107
weighted avg       0.37      0.36      0.37       107

Loaded dataclean_cough_mfcc\136bac9a3e081\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\136bac9a3e081\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\b121e45942a46\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\b121e45942a46\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\6b6853c07e4fb\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\6b6853c07e4fb\mfcc_features.json embedding dim 63
Loaded datac

...

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWa

Loaded dataclean_vowel_mfcc\366890d63fc94\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\c6adb9dc3806d\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\c6adb9dc3806d\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\2a46e5df9bbfa\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\2a46e5df9bbfa\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\4dadb4e93731e\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\4dadb4e93731e\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\004db89f19f9b\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\004db89f19f9b\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\5faa21272abed\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\5faa21272abed\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\63dbec4e28d81\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\63dbec4e28d81\mfcc_features.json emb

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWa

Loaded dataclean_vowel_mfcc\8d886413dd18f\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\171f275cf8e47\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\171f275cf8e47\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\aa510e8941af7\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\aa510e8941af7\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\52dd5c42e836b\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\52dd5c42e836b\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\00da61a101bf6\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\00da61a101bf6\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\19524f5d63516\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\19524f5d63516\mfcc_features.json embedding dim 63
Loaded dataclean_cough_mfcc\b7bb0b4e8b458\mfcc_features.json embedding dim 63
Loaded dataclean_vowel_mfcc\b7bb0b4e8b458\mfcc_features.json emb

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWa